# Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np
import bokeh
import datetime
import math
import itertools
import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all" 

from bokeh.plotting import *
from bokeh.plotting import figure,reset_output
from bokeh.io import output_notebook, show, push_notebook,output_file
from bokeh.models import ColumnDataSource, DateRangeSlider, BoxAnnotation,RangeTool,Whisker
from bokeh.layouts import layout, row, column, Spacer,gridplot,WidgetBox,Row,Column
from bokeh.models import HoverTool,NumeralTickFormatter
from bokeh.io import curdoc
from bokeh.models.widgets import Tabs, Panel,Dropdown,Div,Select,SelectEditor
from bokeh.client import push_session, pull_session
from bokeh.core.properties import value
from bokeh.palettes import Dark2_5 as palette
from bokeh.transform import dodge

import os 
import gc
gc.collect()
path=os.getcwd()
datapath=path+"\SampleInputFiles"
print(path)
os.chdir(datapath)

C:\Users\1025992\Desktop\LearningContent\WiMLDS_BokehSession


# Reading data into Pandas Dataframes

In [2]:
productsdata=pd.read_csv("Products.csv",sep=';',dtype={'ProductID':'str'})
locationsdata=pd.read_csv("Locations.csv",sep=';',dtype={'LocationID':'str'})
salesdata=pd.read_csv("Sales.csv",sep=';',dtype={'ProductID':'str','LocationID':'str'},parse_dates = ['ValidDay'])
bPrice=False
bPredictions=False
bPredictions_ma=False

productsdata['ProdIDDesc']=productsdata['ProductID'].astype(str)+" - "+ productsdata['Name']
productsdata.loc[productsdata['CategoryName'].isnull(),'CategoryName']=productsdata.loc[productsdata['CategoryName'].isnull(),'CategoryID']
sales_mindate= salesdata.ValidDay.min()
sales_maxdate= salesdata.ValidDay.max()
salesdata['Year']=salesdata.ValidDay.map(lambda x:x.year)
salesdata['Month']=salesdata.ValidDay.map(lambda x:x.month)
prodsales_merged=salesdata.merge(productsdata,how="inner",on=['ProductID'],suffixes=('_sales','_prod'))
sales_cut_date = (prodsales_merged.ValidDay>=sales_mindate) & (prodsales_merged.ValidDay<=sales_maxdate)
sales_filtered=prodsales_merged[sales_cut_date].sort_values(by='Quantity', ascending=False)

if os.path.isfile("PriceDayLevel.csv.gz"):
    bPrice=True
    pricedata=pd.read_csv("PriceDayLevel.csv.gz",sep=';',dtype={'ProductID':'str','LocationID':'str'},parse_dates = ['Date'])
    price_cut_date = (pricedata.Date>=sales_mindate) & (pricedata.Date<=sales_maxdate)
    price_filtered=pricedata[price_cut_date]

if os.path.isfile("Predictions.csv.gz"):
    bPredictions=True
    predictionsdata=pd.read_csv("Predictions.csv.gz",sep=';',dtype={'ProductID':'str','LocationID':'str'},parse_dates = ['Date'])

    
if os.path.isfile("Predictions_MA.csv.gz"):
    bPredictions_ma=True
    predictions_ma_data=pd.read_csv("Predictions_MA.csv.gz",sep=';',dtype={'ProductID':'str','LocationID':'str'},parse_dates = ['Date'])

## Statistics of the data read

In [3]:
def returnstats(data,datasetname):
    print("****************************",datasetname , " Details**********************************")
    print("Shape: \n",data.shape,"\n")
    print("Data Types: \n", data.dtypes,"\n")
    print("Missing Values: \n",data.isnull().sum(),"\n")
    print("Sample view of data: \n",data.head(3),"\n")
    
returnstats(productsdata,"Products")
returnstats(locationsdata,"Locations")
returnstats(salesdata,"Sales")
returnstats(pricedata,"Price")

**************************** Products  Details**********************************
Shape: 
 (50, 12) 

Data Types: 
 ProductID             object
ProductDescription    object
Name                  object
UnitID                object
CategoryID            object
CategoryName          object
SubCategoryID         object
SubCategoryName       object
SubSubCategoryID      object
SubSubCategoryName    object
ShelfLife              int64
ProdIDDesc            object
dtype: object 

Missing Values: 
 ProductID             0
ProductDescription    0
Name                  0
UnitID                0
CategoryID            0
CategoryName          0
SubCategoryID         0
SubCategoryName       0
SubSubCategoryID      0
SubSubCategoryName    0
ShelfLife             0
ProdIDDesc            0
dtype: int64 

Sample view of data: 
      ProductID        ProductDescription                      Name UnitID  \
0     20039812  PC BROCCOLI FLORETS 340G  PC BROCCOLI FLORETS 340G     EA   
1  20600792001     ORGA

# Logic to update the combo dropdowns values upon selection of a value from one dropdown

In [4]:
def makedata_dropdownchange(selected_categoryid,is_cat):
    all=('ALL','ALL')
    productslist=[]
    if (selected_categoryid=='ALL'):
            productslist=list(dropdowns_dataset['ProductID'].unique())
    else:                   
        if is_cat:
            categorydata=dropdowns_dataset[dropdowns_dataset['CategoryID'].isin([selected_categoryid])]
            productslist=list(categorydata['ProductID'].unique())
        else:
            productslist=list(dropdowns_dataset['ProductID'].unique())
              
    prodlocdataset=prodloc_dataset[prodloc_dataset['ProductID'].isin(productslist)]
    locations=prodlocdataset['LocationID'].unique()
    locationsmenu=[all]
    locationsmenu.extend(list(zip(locations,locations)))
    
    return(productslist,locationsmenu)

def catg_drpdown_changed(attr, old, new):
    global selected_categoryid,selected_prodid,selected_locid
    selected_categoryid=categories_dropdown.value
    selected_prodid,selected_locid="ALL","ALL"
    products_dropdown.value=selected_prodid     
    locations_dropdown.value=selected_locid
        
    productslist,locationsmenu=makedata_dropdownchange(selected_categoryid,True)
    
    productsmenu=[all]
    productsmenu.extend(list(zip(productslist,productslist)))
    products_dropdown.options=productsmenu     
    locations_dropdown.options=locationsmenu
    
    categorydata=sales_filtered[sales_filtered['ProductID'].isin(productslist)]    
    new_totalprodsalesbyyear,new_salesyear=makedata_prodsalesyearplot(categorydata)
    totalprodsalesbyyear_src.data=ColumnDataSource.from_df(new_totalprodsalesbyyear)
    xvalues=list(new_totalprodsalesbyyear['X'])
    plt_prodsalesbyyear.x_range.factors =xvalues
    
    updateplot_salesweekly(productslist)
    updateplot_prodsaleshistory(productslist,False)
    
def product_drpdown_changed(attr, old, new):
    global selected_categoryid,selected_prodid,selected_locid
    selected_prodid=products_dropdown.value
    
    selected_locid="ALL"
    locations_dropdown.value=selected_locid
    productslist=[]
    
    if selected_prodid=="ALL":
        productslist,locationsmenu=makedata_dropdownchange(selected_categoryid,False)   
    else:
        productslist.append(selected_prodid)
        
    prodlocdataset=prodloc_dataset[prodloc_dataset['ProductID'].isin(productslist)]   
    locations=prodlocdataset['LocationID'].unique()
    locationsmenu=[all]
    locationsmenu.extend(list(zip(locations,locations)))
        
    locations_dropdown.options=locationsmenu
    
    productsdata=sales_filtered[sales_filtered['ProductID'].isin(productslist)] 
    new_totalprodsalesbyyear,new_salesyear=makedata_prodsalesyearplot(productsdata)
    totalprodsalesbyyear_src.data=ColumnDataSource.from_df(new_totalprodsalesbyyear)
    
    xvalues=list(new_totalprodsalesbyyear['X'])
    plt_prodsalesbyyear.x_range.factors =xvalues

    updateplot_salesweekly(productslist)
    updateplot_prodsaleshistory(productslist,False)
    
def location_drpdown_changed(attr, old, new):
    global selected_categoryid,selected_prodid,selected_locid
    selected_locid=locations_dropdown.value
        
    productslist=[]
    if selected_prodid=='ALL':
        prodlist=products_dropdown.options
        productslist=[prodlist[i][1] for i in range(1,len(prodlist))]        
    else:
        productslist.append(selected_prodid)
    
    locationslist=[]
    if selected_locid=='ALL':
        loclist=locations_dropdown.options
        locationslist=[loclist[i][1] for i in range(1,len(loclist))]        
    else:
        locationslist.append(selected_locid)
            
    updateplot_salesweekly(productslist,locationslist)
    if selected_locid=='ALL':
        updateplot_prodsaleshistory(productslist,False,locationslist)
    else:
        updateplot_prodsaleshistory(productslist,True,locationslist)

# Method Definitions for updating data and plots upon selection of values from dropdown and slider

In [5]:
# ## Method Definitions
def makedata_prodsalesyearplot(sales_filtered):          
        groupby_cols=['ProductID','Year']           
        totprodsales=sales_filtered.groupby(groupby_cols).Quantity.sum().reset_index()
        totprodsales.columns=['X','Year','Quantity']
        totprodsales['Quantity']=totprodsales.Quantity.astype(int)
        totprodsales['Year']= totprodsales.Year.astype(str)

        totalprodsalescount= totprodsales.pivot(index = 'X', columns ='Year',values = 'Quantity').reset_index()
        totalprodsalescount=totalprodsalescount.fillna(0)
        totalprodsalescount['X']=totalprodsalescount['X'].astype(str)
        #productslist=list(totalprodsalescount['X'].unique())

        salesyear=list(totalprodsalescount.columns)
        salesyear=["year_"+ str(x) if x!='X' else x for x in salesyear]
        totalprodsalescount.columns = list(salesyear)
        totalprodsalescount.dtypes
        salesyear.remove('X')

        totalprodsalescount['Total']=totalprodsalescount[salesyear].sum(axis = 1, skipna = True)
        totalprodsalescount=totalprodsalescount.sort_values(by='Total',ascending=False)
        return(totalprodsalescount,salesyear) 
    
def makeplot_prodsalesbyyear(p,xlabel):
        p.toolbar.logo = None
        p.y_range.start = 0
        p.x_range.range_padding = 0.1
        
        p.xaxis.axis_label = xlabel
        p.yaxis.axis_label = "TotalSales"
        
        p.xgrid.grid_line_color = None
        p.axis.minor_tick_line_color = None
        p.outline_line_color = None      
        p.yaxis.formatter=NumeralTickFormatter(format="0,0")#"0 a"
        p.legend.location = "top_right"
        p.legend.orientation = "horizontal"
        p.legend.label_text_font_style  = "bold"
        p.legend.label_text_font_size = "8pt"
        p.xaxis.major_label_orientation = math.pi/2
        p.select_one(HoverTool).tooltips = [('ProductID', '@X'),('Year Sales', '@$name'), ('Total Sales', '@Total')]          
        return p
    
def makedata_salesweekly(salesdata,productslist,locationlist=[]):      
    sales_weekly=sales_filtered[sales_filtered['ProductID'].isin(productslist)]
    if locationlist !=[]:
        sales_weekly=sales_filtered[sales_filtered['LocationID'].isin(locationlist)]
    
    sales_weekly['WeekDay']=sales_weekly['ValidDay'].apply(lambda x:str(pd.to_datetime(x).strftime('%A')[:3]))
    salesweekly=sales_weekly[['WeekDay','Quantity']] 
    
    groups = salesweekly.groupby(['WeekDay'])['Quantity'].agg({"Min":"min","Max":"max","Average":"mean","Median":"median","Total":"sum"}).reset_index()
    return(groups)
    
def updateplot_salesweekly(productslist,locationlist=[]):      
    new_groups=makedata_salesweekly(sales_filtered,productslist,locationlist)
    new_groups['height']=np.where(new_groups['Max'].max()<50,0.2,2)
            
    seg1.data_source.data = ColumnDataSource.from_df(new_groups)
    seg2.data_source.data = ColumnDataSource.from_df(new_groups)
    
    rect1.data_source.data = ColumnDataSource.from_df(new_groups)
    rect2.data_source.data = ColumnDataSource.from_df(new_groups)
    rect3.data_source.data = ColumnDataSource.from_df(new_groups)

def makedata_prodlocsaleshist(productslist,isLocation,locationlist=[]):
    global bPrice
    global predictions
    global predictions_ma

    plotprice=False
    prices=pd.DataFrame()
    predictions=pd.DataFrame()
    predictions_ma=pd.DataFrame()
    
    cut1=sales_filtered['ProductID'].isin(productslist)           
    sales=sales_filtered[cut1]
        
    if isLocation:
        sales=sales[sales_filtered['LocationID'].isin(locationlist)]
            
    sales=sales.groupby(['ValidDay']).Quantity.sum().reset_index()    
    sales.ValidDay = pd.to_datetime(sales.ValidDay)
    sales['ValidDayStr']=sales.ValidDay.astype(str)

    if bPrice:
        if isLocation:
            print("##pricedata")
            print(price_filtered.shape)
            
            cut1 = (price_filtered.ProductID.isin(productslist))
            cut2=(price_filtered.LocationID.isin(locationlist))
            price_data=price_filtered[cut1 & cut2].groupby(['ProductID','Type','Date']).CustomerPrice.sum().reset_index()
            price_data.Date = pd.to_datetime(price_data.Date)
            prices= price_data.pivot_table(index = ['ProductID','Date'], columns ='Type',values = 'CustomerPrice',fill_value=0).reset_index()            
            prices['DateStr']=prices.Date.astype(str)
            plotprice=prices.shape[0]>0

        else:
            print("Not a Location", price_filtered.shape)
            plotprice=False
            prices=price_filtered.copy()

    print("Predictions Filtering")
    if bPredictions:
        cut1 = (predictionsdata.ProductID.isin(productslist))
        predictions=predictionsdata[cut1]
        print("After filtering productslist",predictions.shape)     
        if isLocation:
            predictions=predictions[predictions['LocationID'].isin(locationlist)]
            print("After filtering Location list",predictions.shape)
                
        predictions=predictions.groupby(['Date']).yhat.sum().reset_index()
        predictions.Date = pd.to_datetime(predictions.Date)
        predictions['DateStr']=predictions.Date.astype(str) 
        
    if bPredictions_ma:
        #predictionadata
        cut1 = (predictions_ma_data.ProductID.isin(productslist))
        predictions_ma=predictions_ma_data[cut1]
        print("After filtering productslist",predictions_ma.shape)     
        if isLocation:
            predictions_ma=predictions_ma[predictions_ma['LocationID'].isin(locationlist)]
            print("After filtering Location list",predictions_ma.shape)
                
        predictions_ma=predictions_ma.groupby(['Date']).Forecast.sum().reset_index()
        predictions_ma.Date = pd.to_datetime(predictions_ma.Date)
        predictions_ma['DateStr']=predictions_ma.Date.astype(str) 
        
    print("plotprice while plotting:",plotprice)
    return(sales,prices,plotprice,predictions,predictions_ma)
    
def update_start_range(attr, old, new):
    range_tool.x_range.start =new
    p1.x_range.start = new
    p3.x_range.start = new
        
def update_end_range(attr, old, new):
    range_tool.x_range.end =new
    p1.x_range.end = new
    p3.x_range.end = new
            
def makeplot_prodlocsaleshist(sales,price,plotprice,p1,p2,range_tool,predictions,predictions_ma): 
        # Main figure 
        global bPrice
        global bPredictions
        global bPredictions_ma
        p1.toolbar.logo = None
        
        p1.yaxis.axis_label = "Total Sales"
        p1.legend.orientation = "horizontal"
        p1.outline_line_width = 0
        p1.outline_line_alpha = 0
        p1.outline_line_color = None
                
    
        p2.xaxis.axis_label = "Day"
        p2.yaxis.major_label_text_color = None
        p2.yaxis.major_tick_line_color= None
        p2.yaxis.minor_tick_line_color= None
        p2.grid.grid_line_color=None
        
        p1.legend.border_line_width = 0
        p1.legend.border_line_color = None
        p1.legend.border_line_alpha = 0
        p1.legend.label_standoff = 5
        p1.legend.glyph_width = 10
        p1.legend.spacing = 2
        p1.legend.padding = 2
        p1.legend.margin = 2
        
        range_tool.overlay.fill_color = "navy"
        range_tool.overlay.fill_alpha = 0.2

        p2.add_tools(range_tool)
        p2.toolbar.active_multi = range_tool

        # Format the tooltip
        tooltips = [('ValidDay','@ValidDayStr'),('Sales Quantity','@Quantity'),]

        # Configure a renderer to be used upon hover
        hover_glyph = p1.circle(x='ValidDay', y='Quantity', source=sales,size=15, alpha=0,hover_fill_color='black', hover_alpha=0.5)
        
        # Add the HoverTool to the figure
        p1.add_tools(HoverTool(tooltips=tooltips,renderers=[hover_glyph]))

        if bPredictions:
            # Configure a renderer to be used upon hover
            hover_glyph4 = p1.circle(x='Date', y='yhat', source=predictions, size=15, alpha=0,hover_fill_color='black', hover_alpha=0.5)
            # Add the HoverTool to the figure
            p1.add_tools(HoverTool(tooltips=[('ValidDay','@DateStr'),('Predicted Sales','@yhat')],renderers=[hover_glyph4]))
            
        
        if bPredictions_ma:
            # Configure a renderer to be used upon hover
            hover_glyph5 = p1.circle(x='Date', y='Forecast', source=predictions_ma,size=15, alpha=0,hover_fill_color='black', hover_alpha=0.5)
            # Add the HoverTool to the figure
            p1.add_tools(HoverTool(tooltips=[('ValidDay','@DateStr'),('Predicted Sales','@Forecast')],renderers=[hover_glyph5]))
            
            
        range_tool.x_range.on_change('start',update_start_range)
        range_tool.x_range.on_change('end',update_end_range)

        # Plot
        p1.line('ValidDay', 'Quantity', source=sales, color='#006BB6', legend='Sales')           
        if bPredictions:
            p1.line('Date', 'yhat', source=predictions,color='#CE1141', legend='Predictions')
            
        if bPredictions_ma:
            p1.line('Date', 'Forecast', source=predictions_ma,color='darkgoldenrod', legend='Predictions_ma')
        
        p2.line('ValidDay', 'Quantity', source=sales,color='#006BB6')
                                
        layout=column(p1, p2)
        return layout
    
def updateplot_prodsaleshistory(productslist,isLocation,locationlist=[]):
    global bPrice
    global bPredictions
    
    new_prodlocsaleshist,new_prodlocpricehist,new_plotprice,new_predictions,new_predictions_ma=makedata_prodlocsaleshist(productslist,isLocation,locationlist)
    if new_plotprice==True:
        p1.xaxis.major_tick_line_color= None
        p1.xaxis.minor_tick_line_color= None
        p1.xaxis.major_tick_line_width = 0
    else:
        p1.xaxis.major_label_text_color = "black"
        p1.xaxis.major_tick_line_color = "black"
        p1.xaxis.minor_tick_line_color = "black"
            
    prodlocsaleshist_src.data=ColumnDataSource.from_df(new_prodlocsaleshist)
    if bPredictions:
        prodlocpredhist_src.data=ColumnDataSource.from_df(new_predictions)
    
    if bPredictions_ma:
            prodlocpred_mahist_src.data=ColumnDataSource.from_df(new_predictions_ma)
        
    if new_plotprice==True:
        prodlocpricehist_src.data=ColumnDataSource.from_df(new_prodlocpricehist)    
    
    p1.title.text="Total Sales History"
    p1.x_range.start = min(prodlocsaleshist_src.data['ValidDay'])
    p1.x_range.end =  max(prodlocsaleshist_src.data['ValidDay'])     
    p2.x_range.start = min(prodlocsaleshist_src.data['ValidDay'])
    p2.x_range.end =  max(prodlocsaleshist_src.data['ValidDay'])
    range_tool.x_range.start = min(prodlocsaleshist_src.data['ValidDay'])
    range_tool.x_range.end =  max(prodlocsaleshist_src.data['ValidDay'])
    
    listOfSubLayouts = l.children[-1].children
    plotToRemove = curdoc().get_model_by_name('PricePlot')
    if plotToRemove!=None:
        listOfSubLayouts.remove(plotToRemove)
    
    if new_plotprice==True:       
        # Configure a renderer to be used upon hover
        hover_glyph3 = p3.circle(x='Date', y='NORMAL', source=prodlocpricehist_src,
                                size=15, alpha=0,hover_fill_color='black', hover_alpha=0.5)
            
        # Add the HoverTool to the figure
        p3.add_tools(HoverTool(tooltips=[('Date','@DateStr'),('Normal Price','@NORMAL'),],renderers=[hover_glyph3]))
        p3.yaxis.axis_label = "Price"
        p3.outline_line_width = 0
        p3.outline_line_alpha = 0
        p3.outline_line_color = None
        p3.line('Date', 'NORMAL', source=prodlocpricehist_src, color='#008000')
        p3.x_range.start = min(prodlocsaleshist_src.data['ValidDay'])
        p3.x_range.end =  max(prodlocsaleshist_src.data['ValidDay'])
        
        salesplot=l.children[-1].children[0]
        plotrange=l.children[-1].children[-1]
        l.children[-1].children.pop()
                    
        l.children[-1].children.append(p3)
        l.children[-1].children.append(plotrange)

# Main function for the dashboard #################

In [6]:
# ## Actual Code calls
all=('ALL','ALL')

cols=['CategoryID','CategoryName','ProductID','Name','ProdIDDesc']
dropdowns_dataset=sales_filtered.groupby(cols).count().reset_index()
dropdowns_dataset=dropdowns_dataset[cols]

selected_categoryid,selected_prodid,selected_locid="ALL","ALL","ALL"
categoriesdata=dropdowns_dataset
categories_list=categoriesdata.groupby(['CategoryID','CategoryName']).count().reset_index()
categories_list=categories_list[['CategoryID','CategoryName']].astype(str)
categorymenu=[all]
categorymenu.extend(list(zip(categories_list['CategoryID'],categories_list['CategoryName'])))

cols=['ProductID','LocationID']
prodloc_dataset=sales_filtered.groupby(cols).count().reset_index()
prodloc_dataset=prodloc_dataset[cols]
prodloc_dataset[cols]=prodloc_dataset[cols].astype(str)

productslist,locationsmenu=makedata_dropdownchange(selected_categoryid,True)

productsmenu=[all]
productsmenu.extend(list(zip(productslist,productslist)))

prodlocdataset=prodloc_dataset[prodloc_dataset['ProductID'].isin(productslist)]
locations=prodlocdataset['LocationID'].unique()
locationsmenu=[all]
locationsmenu.extend(list(zip(locations,locations)))

categories_dropdown = Select(title="Categories:", value="ALL", options=categorymenu)
products_dropdown = Select(title="Products:", value="ALL", options=productsmenu)
locations_dropdown = Select(title="Locations:", value="ALL", options=locationsmenu)

categories_dropdown.on_change('value',catg_drpdown_changed)
products_dropdown.on_change('value',product_drpdown_changed)
locations_dropdown.on_change('value',location_drpdown_changed)


###########################################plt_prodsalesbyyear ######################################################
totalprodsalesbyyear,salesyear=makedata_prodsalesyearplot(sales_filtered)
totalprodsalesbyyear_src=ColumnDataSource(totalprodsalesbyyear)

tools = "save,hover,box_zoom, reset"    
colors = itertools.cycle(palette)
colorlist=[]
for m, color in zip(range(len(salesyear)), colors):
        colorlist.append(color)
        
xvalues=list(totalprodsalesbyyear['X'])

plt_prodsalesbyyear = figure(x_range=xvalues,min_border_left=50, plot_height = 400,plot_width = 500,
                             title="Product Sales Count by Year", toolbar_location="right", tools=tools)

widthgap= -0.5
for idx,x in enumerate(salesyear):
    yearvalue=salesyear[idx].split('_')[1]
    widthgap+=0.25
    topvalue="year_"+yearvalue
    plt_prodsalesbyyear.vbar(x=dodge('X', widthgap, range=plt_prodsalesbyyear.x_range),top=topvalue,
                    width=0.2, color=colorlist[idx],source=totalprodsalesbyyear_src,legend=[yearvalue],name=topvalue) 

plot_prodsalesbyyear=makeplot_prodsalesbyyear(plt_prodsalesbyyear,"Products")

###########################################plot_salesweekly ######################################################

groups=makedata_salesweekly(sales_filtered,productslist)
tools = "save"
cats=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
plot_salesweekly = figure(min_border_left=80,tools=tools, x_range=cats, toolbar_location="right",plot_height = 400,plot_width = 450,title="Product Sales Weekly")
plot_salesweekly.toolbar.logo = None
plot_salesweekly.xaxis.axis_label = "WeekDay"
plot_salesweekly.yaxis.axis_label = "Sales"
#plot_salesweekly.grid.grid_line_color=None

groups['height']=np.where(groups['Max'].max()<50,0.2,2)

# stems
seg1=plot_salesweekly.segment(x0="WeekDay", y0="Average", x1="WeekDay",y1= "Max", line_color="black",source=groups)
seg2=plot_salesweekly.segment(x0="WeekDay", y0="Min", x1="WeekDay",y1= "Average", line_color="black",source=groups)

# whiskers (almost-0 height rects simpler than segments)
rect1=plot_salesweekly.rect(x="WeekDay",y= "Average",width= 0.2,height="height",  fill_color="#E08E79",line_color="black",source=groups)

# whiskers (almost-0 height rects simpler than segments)
rect2=plot_salesweekly.rect(x="WeekDay",y= "Max",width= 0.2,height= 0.1,line_color="black",source=groups)
rect3=plot_salesweekly.rect(x="WeekDay",y= "Min",width= 0.2,height= 0.1,line_color="black",source=groups)


hover = HoverTool(tooltips = [('WeekDay', '@WeekDay'),('Min Sales', '@Min'),('Average Sales', '@Average'),('Max Sales', '@Max'),('Total Sales','@Total')] ,                  renderers = [rect1,rect2,rect3])
plot_salesweekly.add_tools(hover)

plot_salesweekly.y_range.start = 0
plot_salesweekly.xgrid.grid_line_color = None
plot_salesweekly.x_range.range_padding = 0.2
plot_salesweekly.xaxis.major_label_text_font_size="10pt"
plot_salesweekly.outline_line_color = None


###########################################prodsaleshist_layout ######################################################

prodlocsaleshist,prodlocpricehist,plotprice,plotpredictions,plotpredictions_ma= makedata_prodlocsaleshist(productslist,False)

prodlocsaleshist_src=ColumnDataSource(prodlocsaleshist)
prodlocpricehist_src=ColumnDataSource(prodlocpricehist)
prodlocpredhist_src=ColumnDataSource(plotpredictions)
prodlocpred_mahist_src=ColumnDataSource(plotpredictions_ma)

titledesc="Total Sales History"
tools="save,hover,box_zoom, reset"
plot_width=500
p1 = figure(min_border_left=50, plot_height = 300, plot_width = 950,x_axis_type='datetime',tools=tools, toolbar_location='right',
                x_range=(min(prodlocsaleshist_src.data['ValidDay']), max(prodlocsaleshist_src.data['ValidDay'])), title=titledesc)
            
# Slider figure
p2 = figure(plot_height = 80, plot_width = 950, x_axis_type='datetime', toolbar_location=None,
                x_range=(min(prodlocsaleshist_src.data['ValidDay']), max(prodlocsaleshist_src.data['ValidDay'])))


range_tool = RangeTool(x_range=p1.x_range)  

p3 = figure(plot_height = 120, plot_width = 950, x_axis_type='datetime', toolbar_location=None,
            x_range=(min(prodlocsaleshist_src.data['ValidDay']), max(prodlocsaleshist_src.data['ValidDay'])),name='PricePlot')
p3.grid.grid_line_color=None

prodsaleshist_layout=makeplot_prodlocsaleshist(prodlocsaleshist_src,prodlocpricehist_src,plotprice,p1,p2,range_tool,prodlocpredhist_src,prodlocpred_mahist_src)

############################################## Layout of the dashboard#########################################
useroption_layout=[[row(WidgetBox(categories_dropdown),WidgetBox(products_dropdown), WidgetBox(locations_dropdown))]]
l = layout([[useroption_layout],[row(plot_prodsalesbyyear,plot_salesweekly)],prodsaleshist_layout])#, sizing_mode='scale_width'


show(l)
#curdoc().add_root(l)

GlyphRenderer(id='1038', ...)

GlyphRenderer(id='1054', ...)

GlyphRenderer(id='1071', ...)

Not a Location (6912, 6)
Predictions Filtering
plotprice while plotting: False


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html

